In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [ ]:
""" 数据抽查 sz_max_50 """
def datacheck():
    sum = len(df)
    bins = [-float('inf'), 0, 3, 8, 15, float('inf')] 
    labels = ['0-0', '0-3%', '3-8%',  '8-15%', '>15%']

    df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
    print( df['sz_max_5_bin'].value_counts()/sum, df['sz_max_5'].mean() )

    df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
    print( df['sz_max_10_bin'].value_counts()/sum, df['sz_max_10'].mean() )

    df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
    print( df['sz_max_20_bin'].value_counts()/sum, df['sz_max_20'].mean() )


In [ ]:

# 指标库读出基本数据 
# xl_01dgree = 0.00174533 # 0.1度 
# xl_02dgree = 0.00349067 # 0.2度 
# xl_03dgree = 0.00523604 # 0.3度
# xl_dgree = xl_03dgree
# print( 'd60 斜率：', str(xl_dgree) )

# trade_date like '%2023-06%' and \
# sql1 = "SELECT * \
# from day_zhibiao \
# where \
# jx_dg_250>-0.1 and jx_dg_60>-0.1 and \
# jx_days_ud60>=20 and jx_days_ud60<=40 and \
# jx_dg_20>=0 and jx_zs_20>=1 and pct_fl_3_10>1.5 and \
# (pct_fl_1_3>1.5 or k_red3>=3) "

sql2 = "SELECT * \
from day_zhibiao \
where \
jx_dg_250>0 and jx_dg_60>0 and \
jx_dg_20>=0 and jx_zs_60>=1 and pct_fl_3_10>1.2 and \
(pct_fl_1_3>1.2 or k_red) "


print( sql2 )
df = pd.read_sql_query(sql2, conn)

print('shape: ', df.shape, '\n')
datacheck()

In [ ]:
# 检测数据
col_check = ['symbol', 'trade_date', 'pct_change', 'sz_max_10', \
    'diff', 'dea', \
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_20_60', \
    'sz_max_5','sz_max_20',
     ]
sorting = df.sort_values(['trade_date', 'symbol'], ascending=False)
sorting[df['trade_date']>'2023-07'][col_check]
# sorting.head(5)[col_check]

In [ ]:
""" 数据抽查 sz_max_5 """
sum = len(df)
bins = [-float('inf'), 0, 5, 8, float('inf')] 
labels = ['0-0', '0-5%', '5-8%', '>8%']
df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
# print( df['sz_max_10_bin'].value_counts() )
df['sz_max_5_bin'].value_counts()/sum, df['sz_max_5'].mean()

In [ ]:
""" 数据抽查 sz_max_10 """
# sum = len(df)
# bins = [-float('inf'), 0,3,8, float('inf')] 
# labels = ['0', '0-3', '3-8', '>8']
# df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
# df['sz_max_10_bin'].value_counts(), df['sz_max_10_bin'].value_counts()/sum

In [ ]:
""" 数据抽查 sz_max_20 """
sum = len(df)
bins = [-float('inf'), 0, 5, 12, float('inf')] 
labels = ['<=0', '0-5','5-12', '>12']
df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
df['sz_max_20_bin'].value_counts(), df['sz_max_20_bin'].value_counts()/sum, df['sz_max_20'].mean()

In [ ]:
# 涨幅阶梯化

def jieti_5(df):
    # rate = 0, 1, 2, 3
    if df['sz_max_5']==0: rate = 0        
    if df['sz_max_5']>0 and df['sz_max_5']<=3: rate = 1          
    if df['sz_max_5']>3 and df['sz_max_5']<=8: rate = 2          
    if df['sz_max_5']>8: rate = 3        
    return rate
df['sz_jt_5'] = df.apply(jieti_5, axis=1) 

def jieti_10(df):
    # rate = 0, 1, 2, 3
    if df['sz_max_10']==0: rate = 0        
    if df['sz_max_10']>0 and df['sz_max_10']<=3: rate = 1          
    if df['sz_max_10']>3 and df['sz_max_10']<=8: rate = 2          
    if df['sz_max_10']>8: rate = 3
    return rate
df['sz_jt_10'] = df.apply(jieti_10, axis=1) 

def jieti_20(df):
    # rate = 0, 1, 2
    if df['sz_max_20'] <= 8: rate = 0        
    if df['sz_max_20']>8 and df['sz_max_10']<=15: rate = 1        
    if df['sz_max_20']>15: rate = 2        
    return rate
df['sz_jt_20'] = df.apply(jieti_20, axis=1) 

In [ ]:
""" 临时输出sample """ 
col_sample = ['symbol', 'trade_date', 'pct_change', 'sz_max_10', \
    'diff', 'dea', \
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_20_60', \
    'sz_jt_20',
     ]
df[col_sample].head(100).to_csv('ds_qm.csv', index=False) 

In [ ]:
# 数据监测
# df[ ['sz_max_5', 'sz_jt_5', 'sz_max_10','sz_jt_10', 'sz_max_20', 'sz_jt_20']].head(20)
# df[ ['sz_max_20', 'sz_jt_20']].head(20)

In [ ]:
""" 形成数据集 """

# 0-1类训练目标s：
df['test00'] = df['sz_max_20'] > 10
df['test5_8'] = df['sz_max_5'] > 8
df['test10_10'] = df['sz_max_10'] > 10
df['test20_10'] = df['sz_max_20'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# 多分类训练目标
df['class0'] = df['sz_jt_10']
# 确定输出列  
col_list_ds = ['ts_code', 'symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
    'class0','sz_jt_5','sz_jt_10','sz_jt_20' ]
df_ds = df[col_list_ds].copy()
# 'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
# df_ds[ df_ds['test00'] ].shape, df_ds[ df_ds['test5_8'] ].shape, df_ds[ df_ds['test10_10'] ].shape, df_ds[df_ds['test20_10']].shape, df_ds[df_ds['test20_20']].shape, df_ds[ df_ds['test20_30'] ].shape

In [ ]:
""" 输出到mysql """
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', con=conn, index=False, if_exists='append', chunksize=2000) 

In [ ]:
# name: 数据库中的表名
# con: 与read_sql中相同，数据库连接的驱动
# if_exits： 当数据库中的这个表存在的时候，采取的措施是什么，包括三个值，默认为fail
# fail，若表存在，则不进行数据表写入的相关操作
# replace：若表存在，将数据库表中的数据覆盖；
# append：若表存在，将数据写到原表的后面。
# index：是否将df的index单独写到一列中
# index_label:指定列作为df的index输出，此时index为True
# dtype: 指定列的输出到数据库中的数据类型。字典形式储存：{column_name: sql_dtype}